In [1]:
#Import standard libraries
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta
from plotly.subplots import make_subplots
import seaborn as sns

#Import custom libraries
from plottingUtils import plot_log_candlesticks
from dataUtils import dr, estimate_params

In [12]:
#List of products to be analyzed
indices = ["^GSPC","^STOXX50E",  "^AEX","^DJI", "^IXIC", "^TYX"]
crypto = ["BTC-USD", "ETH-USD", "SOL-USD"]
stocks = ["NVDA", "ASML", "AMD", "TSM"]
etfs = ["2B7K.DE"]
products = indices + etfs

# Fetch the historical data for a certain period and interval
df = yf.download(products, period="1y", interval="1d")

#Plot the candlesticks with a logarithmic scale
plot_log_candlesticks(df, products, remove_gaps=True, normalize=True, log_scale=True)

[*********************100%%**********************]  12 of 12 completed


In [13]:
#Find the relative increase/decrease beween the close price of neighboring intervals
df = dr(df, products)

In [14]:
#Analyze the relative increse/decrease data and estimate the geometruc brownian motion parameters
#Using these parameters an estimate of the expected return, the risk and the risk/expected return ratio is made
data = estimate_params(df, products)

#Display the results in a table with color coding
#TODO: Refactor this into a function
cm_green = sns.light_palette("green", as_cmap=True)
cm_red_pos = sns.light_palette("red", as_cmap=True)
cm_red_neg = sns.light_palette("darkred", as_cmap=True, reverse=True)
data_styled = data.style.background_gradient(cmap=cm_red_pos, subset=["std"])
for green_index in ["mean", "mean-std^2/2", "mean-intrest (E)"]:
    data_styled = data_styled.background_gradient(**{'cmap': cm_green}, subset=[green_index])
for red_index in ["risk (R)", "R/E"]:
    data_styled = data_styled.background_gradient(**{'cmap': cm_red_neg}, subset=[red_index])
display(data_styled)

,mean,std,mean-std^2/2,risk (R),mean-intrest (E),R/E
^GSPC,0.001197,0.007541,0.001169,-0.002448,0.001080,-2.266931
^STOXX50E,0.000827,0.008396,0.000792,-0.002953,0.000703,-4.201362
^AEX,0.000684,0.007428,0.000657,-0.002634,0.000568,-4.639566
^DJI,0.000844,0.006427,0.000823,-0.002164,0.000734,-2.948208
^IXIC,0.001525,0.010139,0.001474,-0.003328,0.001385,-2.403723
^TYX,0.000767,0.014094,0.000668,-0.005248,0.000579,-9.062629
GC=F,0.000553,0.007980,0.000521,-0.002915,0.000432,-6.750621
2B7K.DE,0.000858,0.006333,0.000838,-0.002121,0.000749,-2.832025
NVDA,0.005870,0.029763,0.005427,-0.009170,0.005338,-1.717845
ASML,0.002087,0.020183,0.001884,-0.007052,0.001795,-3.929283


In [5]:
#Code for fitting other distributions than normal distributions. 
#TODO: Refactor

dist1 = sp.stats.t
res1 = sp.stats.fit(dist1, dr, bounds = [(0,len(dr)),(-0.1,0.1), (0,0.1)])
dist2 = sp.stats.norm
res2 = sp.stats.fit(dist2, dr, bounds = [(-0.1,0.1), (0,0.1)])
res = res1
print(res)
print(res.nllf())

fig1 = px.histogram(dr, nbins=20, marginal="box", histnorm="probability density")
fig2 = px.line(x=np.linspace(np.min(dr)*1.1, np.max(dr)*1.1, 500), y=dist.pdf(np.linspace(np.min(dr)*1.1, np.max(dr)*1.1, 500), *res.params))
fig2.update_traces(line_color='red')

fig = make_subplots(rows=2, cols=1, shared_yaxes='all', specs=[[{'type': 'box'}], [{'type': 'histogram'}]])

fig.add_trace(go.Box(fig1.data[1]), row=1, col=1)
fig.add_trace(go.Histogram(fig1.data[0]), row=2, col=1)
fig.add_trace(go.Scatter(fig2.data[0]), secondary_y=False, row=2, col=1)

fig.update_layout(yaxis=dict(domain=[0.8516, 1.0]),
                  yaxis2=dict(domain=[0.0, 0.8316],),
                  xaxis=dict(showticklabels=False),                  
                 )

fig.show()

TypeError: object of type 'function' has no len()